# 개요
> 본 코드는 전체적인 교통카드 데이터와 정류장 데이터에 대한 EDA를 수행한 내용입니다.

# 1. 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# 기타 세팅
# 1. 맷플롯립 한글 폰트 표기
fm.findSystemFonts()
font_location = '/usr/share/fonts/NanumFont/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)

# 열 전부 보기
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [1]:
## 제공 파일 정보
'''
- COMM_CODE
  - 행정동코드
  - 법정동코드
- TB_KTS_DWTCD_: 5권역별 교통카드 데이터
  - TB_KTS_AREA_CODE: 읍면동코드
  - TB_KTS_CARDBG_CODE: 카드구분
  - TB_KTS_TFCMN_CODE: 교통수단구분
  - TB_KTS_USERTYPE_CODE: 이용자유형
  - TB_KTS_TCBO_CODE: 정산사 정보
- TB_KTS_ROUTE: 노선
- TB_KTTS_ROUTESTTN: 노선정류장
- TB_KTS_STTN: 정류장
- TB_KTS_USERTYPE_CODE: 이용자 구분 8개 유형
'''

'\n- COMM_CODE\n  - 행정동코드\n  - 법정동코드\n- TB_KTS_DWTCD_: 5권역별 교통카드 데이터\n  - TB_KTS_AREA_CODE: 읍면동코드\n  - TB_KTS_CARDBG_CODE: 카드구분\n  - TB_KTS_TFCMN_CODE: 교통수단구분\n  - TB_KTS_USERTYPE_CODE: 이용자유형\n  - TB_KTS_TCBO_CODE: 정산사 정보\n- TB_KTS_ROUTE: 노선\n- TB_KTTS_ROUTESTTN: 노선정류장\n- TB_KTS_STTN: 정류장\n- TB_KTS_USERTYPE_CODE: 이용자 구분 8개 유형\n'

In [ ]:
# 노선 데이터
route = pd.read_csv('import_data/TB_KTS_ROUTE/202401/TB_KTS_ROUTE_20240102.csv')
# 노선정류장 데이터
route_station = pd.read_csv('import_data/TB_KTS_ROUTESTTN/202401/TB_KTS_ROUTESTTN_20240102.csv')
# 정류장 데이터
station = pd.read_csv('import_data/TB_KTS_STTN/202401/TB_KTS_STTN_20240102.csv')

# 수도권 1월2일 승하차 데이터
sample_df = pd.read_csv('import_data/TB_KTS_DWTCD_METROPOLITAN/202401/TB_KTS_DWTCD_METROPOLITAN_20240102.csv')
sample_df.head()

# 2. 기초 데이터 EDA

## 2.1. 컬럼 속성 분석
- Categorical variants: 운행일자, 정산사ID, 가상카드번호, 정산지역코드, 카드구분코드, 정산사차량ID, 교통수단코드, 정산사노선ID, 승차일시, 정산사승차정류장ID, 하차일시, 정산사하차정류장ID, 트랜잭션ID, 이용자유형코드(시스템)
- Numerical variants: 이용자수, 이용거리, 탑승시간

In [ ]:
# 컬럼 속성 분석 함수
def column_profile(df):
    profile = pd.DataFrame(columns=[
        '데이터타입', '총건수', '유일값수', 'NULL건수', '공백건수',
        '최대빈도값', '최대빈도수', '최소빈도값', '최소빈도수',
    '평균', '표준편차', 'min', 'q1', 'median', 'q3', 'Max'])
    for col in df.columns:
        dtype = df[col].dtype
        total_count = df[col].count()
        unique_count = df[col].nunique()
        null_count = df[col].isnull().sum()
        blank_count = (df[col] == '').sum() if df[col].dtype == 'object' else 0
        zero_count = (df[col] == '0').sum() if df[col].dtype == 'object' else 0

        # 최소, 최대 빈도값
        value_counts = df[col].value_counts(dropna=True)
        mode_value = value_counts.idxmax() if not value_counts.empty else None
        mode_freq = value_counts.max() if not value_counts.empty else None
        mode_min_value = value_counts.idxmin() if not value_counts.empty else None
        mode_min_freq = value_counts.min() if not value_counts.empty else None

        # 숫자형에 최소/25%/50%/75%/최대값 적용
        if np.issubdtype(dtype, np.number):
            avg = round(df[col].mean(),3)
            std = round(df[col].std(),3)
            min_val = df[col].min()
            q1 = df[col].quantile(0.25)
            median = df[col].median()
            q3 = df[col].quantile(0.75)
            max_val = df[col].max()
        else:
            avg = min_val = q1 = median = q3 = max_val = None

        profile.loc[col] = [
            str(dtype),total_count, unique_count, null_count, blank_count,
            mode_value, mode_freq, mode_min_value, mode_min_freq,
            avg, std, min_val, q1, median, q3, max_val]

    return profile

In [ ]:
# 교통카드 이용내역
column_profile(sample_df)

In [ ]:
# 노선정보
column_profile(route)

In [ ]:
# 노선정류장정보
column_profile(route_station)

In [ ]:
# 정류장정보
column_profile(station)

### 2.1.1. Null값 분석

In [ ]:
# missing value

sns.heatmap(sample_df.isnull(), cbar=False)

In [ ]:
sample_df.isnull().sum()

In [ ]:
# 교통수단코드를 엮어서 설명
# 200번대 교통수단코드: 지하철 -> 지하철이 차량ID를 제공하지 않고 있음
carid_null = pd.DataFrame()
carid_null['ratio'] = sample_df.groupby('교통수단코드')['carid_isnull'].mean().sort_values(ascending=False)
carid_null['count'] = sample_df.groupby('교통수단코드')['carid_isnull'].count().sort_values(ascending=False)
carid_null[carid_null['ratio']>0]

In [ ]:
sample_df.교통수단코드.sort_values().unique() # 80개

In [ ]:
# 시각화
fig, ax1 = plt.subplots()

ax1.bar(carid_null.index, carid_null['ratio'], color ='blue', label='NULL rate')
ax1.set_ylabel('Null Rate')

ax2 = ax1.twinx()
ax2.bar(carid_null.index, carid_null['count'], color='red', linestyle='--', label='Total count')
ax2.set_ylabel('Total Count')
ax2.ticklabel_format(axis='y', style='plain')
plt.tight_layout()
plt.show()

In [ ]:
sample_df.describe()

In [ ]:
# 데이터 이상치 유무 ( 승차보다 하차가 앞서는 경우 )
sample_df[(sample_df['하차일시'] < sample_df['승차일시']) & (sample_df.정산사승차정류장ID != sample_df.정산사하차정류장ID)]

In [ ]:
 #이용 빈도수 체크
freq = sample_df['가상카드번호'].value_counts().value_counts()
# 가중 평균
weighted_avg = np.sum(freq.index.to_numpy() * freq.values) / freq.values.sum()
weighted_avg

In [ ]:
# 개인단위 식별 가능한지
sample_df[sample_df['가상카드번호'] == 'zy3hZeIBq+/lka8Jo2GL0JeRKV6ZY/OiKwNLvnmrU+4=']

In [ ]:
# 시간대별 집계
sample_df_time = sample_df.groupby(sample_df['승차일시'].astype(str).str[:10]).agg({'이용자수':'sum', '이용거리':'mean', '탑승시간':'mean'})
sample_df_time

In [ ]:
sample_df.dtypes

In [ ]:
# 1. 운행일자 논리적 일관성(승차일시와 동일한지 여부 확인)
# 승차일시 결측치 인 6건 제외하고 없음
sample_df[~(sample_df['승차일시'].astype(str).str[:8].isin(['20240102', '20240103']))]

In [ ]:
# 2. 가상카드번호: 범주형 변수
# 패턴 검증 - 정합성 - 길이의 최소 최대
# sample_df[sample_df.가상카드번호.str.len()
print('최대값: ', sample_df.가상카드번호.str.len().max())
print('최소값: ', sample_df.가상카드번호.str.len().min())
print('길이 분포 확인: ', sample_df.가상카드번호.str.len().value_counts())

In [ ]:
# 3. 정산지역코드
# 문자열 패턴: 알파벳이 붙은 값들은 어떤 것인지? 시군구코드랑 다른 것인지? 지역 관련 조인 테이블 확인해보기
# print('문자열 길이 분포:', sample_df.정산지역코드.str.len().value_counts())
print(sample_df.loc[sample_df.정산지역코드.str.len() == 5, '정산지역코드'].unique())
print(sample_df.loc[sample_df.정산지역코드.str.len() == 10, '정산지역코드'].unique())

In [ ]:
# 승하차시간 범위
# sample_df[sample_df['운행일자'].unique()
# sample_df['운행일자'].unique()
# 분이나 초의 10의 자리가 5이상인경우
nonull_df = sample_df.dropna(subset='승차일시')
nonull_df[(nonull_df['승차일시'].astype(str).str[10].astype(int)>5) | (nonull_df['승차일시'].astype(str).str[12].astype(int)>5)] # -> None

In [ ]:
nonull_df = sample_df.dropna(subset='하차일시')
nonull_df[(nonull_df['하차일시'].astype(str).str[10].astype(int)>5) | (nonull_df['하차일시'].astype(str).str[12].astype(int)>5)] # -> None

In [ ]:
nonull_df.dtypes

In [ ]:

plt.figure(figsize=(20,4))
plt.plot(sample_df_time.index, sample_df_time['이용자수'], label ='passenger', color='darkblue')
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45)
plt.show()

## 2.2. 정규성 검사

In [ ]:
# 정규성 검사: kstest, QQ-plot
# 이용자수, 이용거리, 탑승시간
from scipy.stats import kstest
import scipy.stats as stats

def isit_normal(df, colname):
    # 10만개 샘플링
    sample = df[colname].dropna().sample(1000000, random_state=42)

    mean, std = sample.mean(), sample.std()

    # kstest
    stat, p = kstest(sample, 'norm', args=(mean, std))
    print(f"K-S 통계량: {stat:.4f}")
    print(f"p-값: {p:.4f}")
    if p> 0.05:
        print("정규성 따름")
    else:
        print("정규성 따르지 않음")

    # qq-plot
    fig, axs = plt.subplots(1,2, figsize=(12,5))
    stats.probplot(sample, dist='norm', plot=axs[0])
    axs[0].set_title(f"{colname} QQ-Plot")

    # histogram + KDE
    axs[1].hist(sample, bins = 50, edgecolor='blue')
    axs[1].ticklabel_format(style='plain', axis='y')
    axs[1].set_title(f"{colname} Histogram")
    axs[1].set_xlabel("값")
    axs[1].set_ylabel("빈도")

    plt.tight_layout()
    plt.show()

In [ ]:
sample_df.이용자수.sort_values().value_counts()

In [ ]:
# 이용자수 정규성 검정
isit_normal(sample_df, '이용자수')

In [ ]:
# 이용거리 정규성 검정
isit_normal(sample_df, '이용거리')

In [ ]:
# 이용거리 정규성 검정
isit_normal(sample_df, '탑승시간')

## 2.3. 교통카드 이용량 정합성 검증

In [ ]:
    sample_df[sample_df['가상카드번호'] == 'zy3hZeIBq+/lka8Jo2GL0JeRKV6ZY/OiKwNLvnmrU+4=']

In [ ]:
# 전체 OD 통행량 개수
sample_df.groupby(['가상카드번호', '트랜잭션ID']).ngroups

In [ ]:
# 환승건수, 이용자수, 이용거리, 탑승시간 관련하여 box-plot
for col in ['환승건수', '이용자수' ,'이용거리', '탑승시간']:
    plt.figure()
    plt.boxplot(sample_df[col].dropna())
    plt.title(col)
    plt.show()


## 2.4. 노선, 정류장 정합성 검증

### 2.4.1. 노선 정보

In [ ]:
# 노선 - 노선정류장
route_ids = set(list(route.노선ID.unique()))
route_station_ids = set(list(route_station.노선ID.unique()))
print(f'노선 정보 노선ID 수:, {len(route_ids)}건')
print(f'노선정류장 정보 노선ID 수:, {len(route_station_ids)}건')
print(f'노선 정보에만 존재하는 노선ID 수: {len(route_ids - route_station_ids)}건')
print(f'노선 정류장 정보에만 존재하는 노선ID 수: {len(route_station_ids- route_ids)}건')

In [ ]:
# 이정도면 다른건데?
print(f'총 노선ID 행 개수: {len(route)}')
route[route.노선거리<0] # 3개

In [ ]:
route.head(10)

### 2.4.2. 노선정류장 정보

In [ ]:
# 노선 정류장 정보가
route_station_ids = set(list(route_station.정류장ID.unique()))
station_ids = set(list(station.정류장ID.unique()))
print(f'노선정류장 정류장ID 고유값:, {len(route_station_ids)}건')
print(f'정류장 정류장ID 고유값:, {len(station_ids)}건')
print(f'노선정류장 정보에만 존재하는 정류장ID 고유값: {len(route_station_ids - station_ids)}건')
print(f'정류장 정보에만 존재하는 정류장ID 고유값: {len(station_ids- route_station_ids)}건')

In [ ]:
# 노선정류장 정류장에만 존재하는 정류장 어떤 건지 확인
route_merged = route_station.merge(station, how = 'left', on = '정류장ID', indicator=True)
route_merged[route_merged['_merge'] == 'left_only']

In [ ]:
missing_ids = list(route_station_ids -station_ids)
missing_station_ids = list(station_ids - route_station_ids)
missing_ids[:10]

In [ ]:
# 노선정류장 정보에만 존재하는 정류장 저보
route_station[route_station['정류장ID'].isin(missing_ids[:10])].sort_values('정류장ID')

In [ ]:
# 정류장 정보에만 있는 정류장 샘플
station[station['정류장ID'].isin(missing_station_ids[:20])].sort_values('정류장ID')

In [ ]:
# 노선 누적거리, 정류장 거리 음수인 경우 체크
print(len(route_station))
route_station[route_station['노선누적거리']<0] # 68게
route_station[route_station['정류장거리']<0] # 1705게

route_station[route_station.노선ID == 31000001].sort_values('정류장순번')

In [ ]:
route_station[route_station.정류장Y좌표 == 0]

In [ ]:
routestation.isnull().sum()

In [ ]:
# 지리적 표시
from shapely.geometry import Point

routestation['geometry'] = routestation.apply(lambda row: Point(row['정류장X좌표'], row['정류장Y좌표']), axis=1)
gdf = gpd.GeoDataFrame(routestation, geometry='geometry')

gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
gdf.plot(markersize=0.5)

In [ ]:
# 좌표 이상치 확인 - 정류장 -> 이상치 값 존재 안함
outliers = station[(station['정류장GPSX좌표'] < 124)|(station['정류장GPSX좌표'] >132)|(station['정류장GPSY좌표']<33)|(station['정류장GPSY좌표']>38.9)]
outliers[(outliers['정류장GPSX좌표'] == 0) | (outliers['정류장GPSY좌표'] == 0)]

In [ ]:
# 좌표 이상치 확인 - 노선정류장
outliers = gdf[(gdf['정류장X좌표'] < 124)|(gdf['정류장X좌표'] >132)|(gdf['정류장Y좌표']<33)|(gdf['정류장Y좌표']>38.9)]
# outliers[(outliers['정류장X좌표'] == 0) | (outliers['정류장Y좌표'] == 0)]
outliers

In [ ]:
# 노선 정류장 정보 -> 정류장 정보로 보완 가능 여부
merge_test = outliers.merge(station, how='left', on='정류장ID', indicator=True)
merge_test[merge_test._merge != 'both']

In [ ]:
# 좌표 0값인 경우 필터링 - 노선정류장 정보
total_size = len(route_station)
zero_route_station = len(route_station[(route_station['정류장X좌표'] == 0) | (route_station['정류장Y좌표'] ==0)])
print(f'노선정류장 정보 좌표 0값인 경우: {zero_route_station}건')
print(f'전체건수 : {total_size}건')
print(f'비율: {round(zero_route_station/total_size*100 ,2)}%')


In [ ]:
station

In [ ]:
# 좌표 0값인 경우 필터링 - 정류장 정보
total_size = len(station)
zero_station = len(station[(station['정류장GPSX좌표'] == 0) | (station['정류장GPSY좌표'] ==0)])
print(f'노선정류장 정보 좌표 0값인 경우: {zero_station}건')
print(f'전체건수 : {total_size}건')
print(f'비율: {round(zero_station/total_size*100 ,2)}%')

### 2.4.3. 정류장 정보

In [ ]:
# 일별로 어떤게 바뀌는지 체크
station_0101 = pd.read_csv('import_data/TB_KTS_STTN/202401/TB_KTS_STTN_20240101.csv')
station_1231 = pd.read_csv('import_data/TB_KTS_STTN/202412/TB_KTS_STTN_20241231.csv')

In [ ]:
print(station_0101.describe())
print('----------------------')
print(station_1231.describe())

In [ ]:
# x,y 좌표 기준 중복 검증
station

## 2.5. 참조 무결성 검증
---
- COMM_CODE
  - TB_ADMIN_DONG_CODE_2024: 행정동코드
  - TB_LAW_DONG_CODE_2024: 법정동코드
- TB_KTS_DWTCD_: 5권역별 교통카드 데이터
  - TB_KTS_AREA_CODE: 읍면동코드
  - TB_KTS_CARDBG_CODE: 카드구분
  - TB_KTS_TFCMN_CODE: 교통수단구분
  - TB_KTS_USERTYPE_CODE: 이용자유형
  - TB_KTS_TCBO_CODE: 정산사 정보
- TB_KTS_ROUTE: 노선
- TB_KTTS_ROUTESTTN: 노선정류장
- TB_KTS_STTN: 정류장
- TB_KTS_USERTYPE_CODE: 이용자 구분 8개 유형

In [ ]:
# 참조 테이블 IMPORT
admin_dong = pd.read_csv('import_data/COMM_CODE/TB_ADMIN_DONG_CODE_2024.csv')
law_dong = pd.read_csv('import_data/COMM_CODE/TB_LAW_DONG_CODE_2024.csv')
company_info = pd.read_csv('import_data/TB_KTS_DWTCD_METROPOLITAN/TB_KTS_TCBO_CODE.csv')

In [ ]:
# 참조 무결성 검증 대상 컬럼

# 1. 정산사ID - 정산사 정보
# 2. 정산지역코드 - 행정동코드, 법정동코드
# 3. 정산사노선ID - 노선, 노선정류장
# 4. 정산사승차정류장ID, 정산사하차정류장ID - 노선정류장, 정류장
# 5. 이용자코드(시스템) - 이용자 구분

# 1. 정산사ID - 정산사 정보
company_merge = sample_df.merge(company_info, how='left', left_on='정산사ID', right_on ='정산사코드', indicator=True)
company_merge[company_merge['_merge'] != 'both'] # -> None 결합 무결

### 2.5.1. 교통카드 이용데이터 결합 무결성 검증

**1) 교통카드 + 정류장**

In [ ]:
# 교통카드 이용데이터(승차) - 정류장 데이터 결합 무결성 검증
station_no_dupe = station.drop_duplicates(subset = '정류장ID') # 결합 테스트를 위해 정류장을 고유값으로 정리
board_station_merge = sample_df.merge(station_no_dupe, how='left', left_on = '정산사승차정류장ID', right_on='정류장ID', indicator=True)

# 결합안된 건수 확인
not_merged = board_station_merge[board_station_merge._merge == 'left_only']
print('승차데이터 결합 안된 건: ', len(not_merged), '건', '\n비율: ', round((len(not_merged) / len(board_station_merge)) * 100,2), '%')
print('merge 정보: ', board_station_merge._merge.value_counts(), '\n-----')

# 결합안된 건 고유값 개수 확인
total_ids = sample_df.정산사승차정류장ID.nunique()
not_merged_ids = not_merged.정산사승차정류장ID.nunique()

print(f'고유값 결합 결과: \n 결합 안된 정류장 고유값 건수: {not_merged_ids}건 \n 전체 정류장 고유값 건수: {total_ids}건 \n 비율: {round(not_merged_ids/total_ids*100, 2)}%')

In [ ]:
# 교통카드 이용데이터(하차) - 정류장 데이터 결합 무결성 검증
alight_station_merge = sample_df.merge(station_no_dupe, how= 'left', left_on = '정산사하차정류장ID', right_on='정류장ID', indicator=True)
not_merged = alight_station_merge[alight_station_merge._merge == 'left_only']
print('하차데이터 결합 안된 건: ', len(not_merged), '건', '\n비율: ', round((len(not_merged) / len(alight_station_merge)) * 100,2), '%')
print('merge 정보: ', alight_station_merge._merge.value_counts(), '\n-----')

# 결합안된 건 고유값 개수 확인
total_ids = sample_df.정산사하차정류장ID.nunique()
not_merged_ids = not_merged.정산사하차정류장ID.nunique()

print(f'고유값 결합 결과: \n 결합 안된 정류장 고유값 건수: {not_merged_ids}건 \n 전체 정류장 고유값 건수: {total_ids}건 \n 비율: {round(not_merged_ids/total_ids*100, 2)}%')

**2) 교통카드 + 노선정류장**

In [ ]:
# 교통카드 이용데이터(승차) - 정류장 데이터 결합 무결성 검증
routestation_no_dupe = route_station.drop_duplicates(subset = '정류장ID') # 결합 테스트를 위해 정류장을 고유값으로 정리
board_station_merge = sample_df.merge(routestation_no_dupe, how='left', left_on = '정산사승차정류장ID', right_on='정류장ID', indicator=True)

# 결합안된 건수 확인
not_merged = board_station_merge[board_station_merge._merge == 'left_only']
print('승차데이터 결합 안된 건: ', len(not_merged), '건', '\n비율: ', round((len(not_merged) / len(board_station_merge)) * 100,2), '%')
print('merge 정보: ', board_station_merge._merge.value_counts(), '\n-----')

# 결합안된 건 고유값 개수 확인
total_ids = sample_df.정산사승차정류장ID.nunique()
not_merged_ids = not_merged.정산사승차정류장ID.nunique()

print(f'고유값 결합 결과: \n 결합 안된 정류장 고유값 건수: {not_merged_ids}건 \n 전체 정류장 고유값 건수: {total_ids}건 \n 비율: {round(not_merged_ids/total_ids*100, 2)}%')

In [ ]:
# 교통카드 이용데이터(하차) - 정류장 데이터 결합 무결성 검증
alight_station_merge = sample_df.merge(routestation_no_dupe, how= 'left', left_on = '정산사하차정류장ID', right_on='정류장ID', indicator=True)
not_merged = alight_station_merge[alight_station_merge._merge == 'left_only']
print('하차데이터 결합 안된 건: ', len(not_merged), '건', '\n비율: ', round((len(not_merged) / len(alight_station_merge)) * 100,2), '%')
print('merge 정보: ', alight_station_merge._merge.value_counts(), '\n-----')

# 결합안된 건 고유값 개수 확인
total_ids = sample_df.정산사하차정류장ID.nunique()
not_merged_ids = not_merged.정산사하차정류장ID.nunique()

print(f'고유값 결합 결과: \n 결합 안된 정류장 고유값 건수: {not_merged_ids}건 \n 전체 정류장 고유값 건수: {total_ids}건 \n 비율: {round(not_merged_ids/total_ids*100, 2)}%')

In [ ]:
# 중복 안되는 키 찾기
print(sample_df.columns) # 운행일자, 정산지역카드, 교통수단코드, 정산사노선ID, 정류장ID
print(station.columns)  # 운행일자, 지역코드, 교통수단구분, 정류장ID,
print(route_station.columns) # 운행일자, 정산지역코드, 노선ID, 교통수단구분, 정류장ID

In [ ]:
sample_df.교통수단코드.unique()
route_station.교통수단구분.unique()

In [ ]:
# 교통카드 이용량 + 노선정류장
print('기존 교통카드 이용량 행 수: ', len(sample_df), '행')
sample_df['교통수단코드'] = sample_df['교통수단코드'].astype(str)
sample_route_station = sample_df.merge(duplicated, how='left',
                left_on =['운행일자', '정산지역코드', '정산사노선ID', '정산사승차정류장ID'],
                right_on = ['운행일자', '정산지역코드', '노선ID', '정류장ID'], indicator=True)
len(sample_route_station)

In [ ]:
# 중복 여부 확인
print('기존 노선정류장정보 행 수: ', len(route_station), '행')
duplicated = route_station.groupby(['운행일자', '노선ID', '정류장ID',  '정산지역코드']).agg({'정류장X좌표':'first', '정류장Y좌표':'first'}).reset_index()
duplicated

In [ ]:
sample_route_station._merge.value_counts()

In [ ]:
# 중복 여부 확인
duplicated = route_station[route_station.duplicated(subset=['운행일자', '노선ID', '정류장ID',  '정산지역코드','정류장순번'], keep=False)].sort_values(['정류장ID', '노선ID'])
duplicated

In [ ]:
# 수도권 이외 권역들에 대한 참조 무결성 검증
# 파일로드
sample_cc = pd.read_csv('import_data/TB_KTS_DWTCD_CHUNGCHEONG/202401/TB_KTS_DWTCD_CHUNGCHEONG_20240102.csv')
sample_gs = pd.read_csv('import_data/TB_KTS_DWTCD_GYEONGSANG/202401/TB_KTS_DWTCD_GYEONGSANG_20240102.csv')
sample_jj = pd.read_csv('import_data/TB_KTS_DWTCD_JEJU/202401/TB_KTS_DWTCD_JEJU_20240102.csv')
sample_jl = pd.read_csv('import_data/TB_KTS_DWTCD_JEOLLA/202401/TB_KTS_DWTCD_JEOLLA_20240102.csv')

In [ ]:
station

In [ ]:
sample_cc.head(3)

In [ ]:
#
print(station.dtypes)
print('-----')
print(sample_cc.dtypes)

In [ ]:
# 교통카드코드 200번대 일경우 지하철 그 외 버스 처리
sample_df['교통수단구분'] = sample_df['교통수단코드'].apply(lambda x: 'T' if (199<x & x<300) else 'B')
sample_cc['교통수단구분'] = sample_cc['교통수단코드'].apply(lambda x: 'T' if ((199<x<300) or (x in [440, 699, 547, 552, 850, 851, 852, 853, 870, 871, 872, 877])) else 'B')
sample_gs['교통수단구분'] = sample_gs['교통수단코드'].apply(lambda x: 'T' if ((199<x<300) or (x in [440, 619, 699, 547, 552, 850, 851, 852, 853, 870, 871, 872, 877])) else 'B')
sample_jj['교통수단구분'] = sample_jj['교통수단코드'].apply(lambda x: 'T' if (199<x & x<300) else 'B')
sample_jl['교통수단구분'] = sample_jl['교통수단코드'].apply(lambda x: 'T' if ((199<x<300) or (x in [4, 440, 699, 547, 552, 850, 851, 852, 853, 870, 871, 872, 877])) else 'B')

In [ ]:
# 지역코드 str
sample_df.정산지역코드 = sample_df.정산지역코드.astype(str)
sample_cc.정산지역코드 = sample_cc.정산지역코드.astype(str)
sample_gs.정산지역코드 = sample_gs.정산지역코드.astype(str)
sample_jj.정산지역코드 = sample_jj.정산지역코드.astype(str)
sample_jl.정산지역코드 = sample_jl.정산지역코드.astype(str)

In [ ]:
station[station.duplicated(['지역코드', '정류장ID', '교통수단구분'], keep =False)]

In [ ]:
print(len(sample_jl))

In [ ]:
test_cc = sample_jl.merge(station, how = 'left',
                left_on =['정산사ID', '정산지역코드', '교통수단구분', '정산사승차정류장ID'],
                right_on = [ '정산사코드', '지역코드', '교통수단구분', '정류장ID'],
                indicator = True)
test_cc._merge.value_counts()

In [ ]:
test_cc[(test_cc._merge == 'left_only') & (~test_cc.정산사승차정류장ID.isnull()) & (~test_cc.정산사하차정류장ID.isnull())]

In [ ]:
station[station['정류장ID'] == 7121004500]
# route_station[(route_station['노선ID'] == 0) &(route_station['정류장ID'] == 8009)]

In [ ]:
print(len(sample_cc))
print(round(973966/len(sample_cc)*100, 2), '%')

In [ ]:
print(len(sample_df))
print(round(16862950/len(sample_df)*100, 2), '%')

In [ ]:
print(len(sample_jl))
print(round(595093/len(sample_jl)*100, 2), '%')

In [ ]:
print(len(sample_gs))
print(round(3726821/len(sample_gs)*100, 2), '%')

In [ ]:
# total
merged = 16862950+973966+3726821+160047+572549
total = len(sample_df) + len(sample_cc) + len(sample_jl) + len(sample_jj) + len(sample_gs)
print('total: ', total, 'merged: ', merged, 'ratio: ', round(merged/total*100,2))

In [ ]:
test_cc[test_cc['_merge'] == 'left_only'].iloc[30:50, :]

### 2.5.2. 하차데이터와의 참조 무결성 검증

In [ ]:
# sample data 추출
sample = sample_df.iloc[:100000, :]
sample

In [ ]:
# 하차 정류장 Null값 비율 21190개
sample.dropna(subset='정산사하차정류장ID', inplace=True) # 21190개 21.2%

In [ ]:
# 교통수단 구분 추가 및 전처리
sample['교통수단구분'] = sample['교통수단코드'].apply(lambda x: 'T' if 199<x<300 else 'B')
sample['정산지역코드'] = sample['정산지역코드'].astype(str)

In [ ]:
station.head(3)

In [ ]:
test_merge = sample.merge(station, how = 'left', left_on = ['정산지역코드', '정산사하차정류장ID', '교통수단구분'], right_on =['지역코드', '정류장ID', '교통수단구분'], indicator=True)
test_merge._merge.value_counts()

In [ ]:
438/78372*100